In [1]:
import netCDF4 as nc
from axikernels.core.handlers.element_output import ElementOutput
import yaml
from scipy.interpolate import interp1d, RegularGridInterpolator
import numpy as np

In [2]:
simulation = ElementOutput('data/3D_KERNEL_EXAMPLE_30s/output/elements')

In [12]:
simulation._load_material_property(points = [[6371000, 0, 180],
                                             [5000000, 0, 0],
                                             [4000000, 0, 0],
                                             [3000000, 0, 0]], material_property='vs')

array([3927.93594269, 6002.45342286, 6605.6751051 ,    0.        ])

In [4]:
with open(simulation.inparam_model, 'r') as file:
    data = yaml.safe_load(file)

material_properties = {}
for model_dict in data['list_of_3D_models']:
    model = list(model_dict.keys())[0]
    material_properties[model] = []
    for material_property in model_dict[model]['properties']:
        material_properties[model].append(list(material_property.keys())[0])
print(material_properties)

{'EMC_S362ANI': ['VP', 'VS']}


In [5]:
model_3d = nc.Dataset('S362ANI_percent.nc', 'r')
# Get the depth, latitude, and longitude arrays
depth = model_3d.variables['depth'][:]
radii = (6371 - depth) * 1e3
latitude = model_3d.variables['latitude'][:]
longitude = model_3d.variables['longitude'][:]

# Get the 'dvs' data
dvs_data = model_3d.variables['dvs'][:]


In [6]:
i, j, k = 0, 0, 0
print(radii[i], latitude[j], longitude[k])
print(dvs_data[i, j, k])

6346000.0 -90.0 -180.0
-0.2925


In [7]:
i, j, k = 1, 0, 0
print(radii[i], latitude[j], longitude[k])
print(dvs_data[i, j, k])

6321000.0 -90.0 -180.0
0.4691


In [8]:
# Define the point where you want to interpolate
point = np.array([3481000, -90, -180])  # Replace with your actual depth, latitude, and longitude

In [9]:
# Create a 3D mesh grid
radii_grid, latitude_grid, longitude_grid = np.meshgrid(radii, latitude, longitude, indexing='ij')

# Calculate the Euclidean distance between the given point and all points in the mesh
distances = np.sqrt((radii_grid - point[0])**2 + (latitude_grid - point[1])**2 + (longitude_grid - point[2])**2)

# Find the indices of the point with the smallest distance
closest_point_indices = np.unravel_index(np.argmin(distances), distances.shape)

# Get the 'dvs' value at the closest point
closest_dvs_value = dvs_data[closest_point_indices]

print(closest_dvs_value)

1.168


In [10]:
def extrapolate_3d(point, radii, latitude, longitude, dvs_data):
    # Create interp1d objects for each dimension
    radii_interpolator = interp1d(radii, dvs_data, axis=0, fill_value="extrapolate")
    latitude_interpolator = interp1d(latitude, radii_interpolator(point[0]), axis=0, fill_value="extrapolate")
    longitude_interpolator = interp1d(longitude, latitude_interpolator(point[1]), axis=0, fill_value="extrapolate")

    # Return the interpolated and extrapolated value
    return longitude_interpolator(point[2])

# Interpolate and extrapolate the 'dvs' value at the point
dvs_at_point = extrapolate_3d(point, radii, latitude, longitude, dvs_data)

print(dvs_at_point)

1.1679999828338623
